<h2> Kamera Calibratsiyasi </h2>
Arzon kameralarning aksariyatida o'ziga yarasha distortion (buzilish) bo'ladi, lekin siz buni ishlatayotganingizda sezmaysiz chunki buni software tomondan to'g'irlasa bo'ladi. Quyidagi tasvirda radial distortionni ko'rishiz mumkin.
<img src="jpg/calib_radial.jpg" alt="projection"/> 

Yuqoridagi distortion quyidagicha yo'qotilsa bo'ladi:

<img src="jpg/k_undis.png" alt="projection"/> 

Shunga o'xshash yana bir distortion bu tangential distortion dir, u qachonki image plan linza bilan bir to`g`ri chiziqda va parallel joylashmagan holatda sodir bo`ladi. Uni quyidagi formula orqali hal qilsa bo`ladi.

<img src="jpg/p_undistor.png" alt="projection"/> 

Shunday qilib bizga 5 ta parameter kerak : 
<img src="jpg/distors.png" alt="projection"/> 


Ishlab chiqarishda va amaliyotda qo`llashda bu parametrlardan boshqa bizga kerak bo`ladigan parameterlar ham bor, ular intrinsic va extrinsic parameterlardir. Intrinsic parameter odatda kamera parameteri deb ham ataladi chunki u parameter faqat kameraga tegishli ular focal uzunligi, va optik markazi. Uni bir marta aniqlab olganimizdan keyin xohlagancha ishlatsak bo'ladi:

<img src="jpg/intrinsic.png" alt="projection"/> 

Extrinsic parameterlar ga esa rotation (qayrilish) va translation (surilish) parameterlar kiradi. 

Bu parameterlarni topish uchun bizga ichidagi edge(bir rangdan ikkinchi rangga o'tish joylari) yaxshi ifodalangan rasmlar kerak bo`ladi. Bu parameterlarni topish uchun bizga 10 ta rasm kerak bo`ladi.

references:
https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_calib3d/py_calibration/py_calibration.html


<b>Kamera geometriyasi</b>

<b>World kordinatasidan image kordinatasiga o`tish</b>

Avvalambor biz oddiy kamerada tasvirning qanday hosib bo'lishidan boshlaymiz, quyidagi rasmda tasvirning kamera planida qanday hosil bo'lishi ko'rsaatilgan. 
<img src="jpg/1200px-Pinhole-camera.svg.png" alt="projection"/> 
Bu jarayon Pinhole camera model deb ataladi.
Computer vision da esa bu modeldan foydalanilmaydi, teskari tasvirning oldini olish uchun kameraning oldida image plan bor deb tasavvur qilinadi va tasvir unda xosil qilinadi, ushbu xolatda focal plane bilan image planni adashtirib yubormang:

<img src="jpg/pinholeCamera.png" alt="projection"/> 

http://aviation_dictionary.enacademic.com/2994/focal_plane


World kordinatasidan image kordinatasiga o'tish uchun odatda birinchi novbatda world coordinatasidan camera coordinatasiga o'tiladi, keyin camera coordinatasidan image coordinatasiga o'tiladi. 

<img src="jpg/projection.png" alt="projection"/> 

<p>Bu yerda X<span style="font-size: 11.6667px;">s</span>, Y<sub>s</sub> , Z<sub>s</sub> lar scene (focal) kordinata sistemasi, X<sub>c</sub>, Y<sub>c</sub> , Z<sub>c</sub> esa kamera kordinatasini bildiradi</p>
<p>World plan dan image planga o`tish uchun bizga kamerani intrinsic parametrlari kerak bo`ladi. Ushbu ma`lumotlar camera callibration orqali olinadi. K = Kamera matrix. Shunday qilib transformation oddiygina linear algebra dagi matritsalar ko`paytmasi orqali amalga oshiriladi:
<br />World plandan kamera planga o`tish:<br />P = K[R|t]<br />
<img src="jpg/world_camera.JPG" alt="projection"/> 

bu yerda t transition matrix yani kordinatarl bo`yicha qancha surilishini ko`rsatadi.

To`liq jarayon yani dunyo (world) kordinatasidan image (rasm) kordinatasiga o`tish:<br />O<sub>img</sub>= P O<sub>w</sub>&nbsp;= K[R|t]O<sub>w</sub></p>
<img src="jpg/world_image.JPG" alt="projection"/> 
<p>Bu yerda K ning hajmi [3x3],[R|t] ning hajmi esa [3x4] ga teng ularning ko`paytmasi esa [3x4] bo`ladi, ammo Ow ning hajmi esa [Xs, Ys , Zs]<sup>T</sup> yani [3x1] ga teng yani ko`paytma amalga oshmaydi biz unga 1 ni qo`shib qo`yamiz ya`ni[Xs, Ys , Zs, 1]<sup>T</sup>&nbsp;qilib va ushbu muammoni oldini olamiz. yani wi = 1</p>

https://openmvg.readthedocs.io/en/latest/openMVG/cameras/cameras/

<b> !! eslatma bu yerda f focal length bo`lib u fizikadagi focal uzunlikni emas balkim image plan bilan focal plane orasidagi masofani ifodalaydi </b>

<h2> CODING </h2>

In [1]:
# Boshlanishidan aytib qo`yay kod https://github.com/udacity/CarND-Camera-Calibration dan uxlatildi
# This code is taken from https://github.com/udacity/CarND-Camera-Calibration
# ishlatib ko`rish uchun o`zingiz chessboardni rasmga tushuring yoki yuqoridagi urldan yuklab oling yoki opencv papkasiga kirib chess degan papkani toping

import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt


In [5]:


# Bu yerda biz 8 x 6 gridli shaxmat doskasi (chessboard) ishlatayotganimiz uchun
# ularning kordinatasini gridlar bo`yicha qo`yib chiqamiz : (0,0,0), (1,0,0), (2,0,0) ....,(7,5,0)
objp = np.zeros((6*8,3), np.float32)
print(np.shape(objp))
objp[:,:2] = np.mgrid[0:8, 0:6].T.reshape(-1,2)
print(np.shape(objp))

print(objp)

(48, 3)
(48, 3)
[[0. 0. 0.]
 [1. 0. 0.]
 [2. 0. 0.]
 [3. 0. 0.]
 [4. 0. 0.]
 [5. 0. 0.]
 [6. 0. 0.]
 [7. 0. 0.]
 [0. 1. 0.]
 [1. 1. 0.]
 [2. 1. 0.]
 [3. 1. 0.]
 [4. 1. 0.]
 [5. 1. 0.]
 [6. 1. 0.]
 [7. 1. 0.]
 [0. 2. 0.]
 [1. 2. 0.]
 [2. 2. 0.]
 [3. 2. 0.]
 [4. 2. 0.]
 [5. 2. 0.]
 [6. 2. 0.]
 [7. 2. 0.]
 [0. 3. 0.]
 [1. 3. 0.]
 [2. 3. 0.]
 [3. 3. 0.]
 [4. 3. 0.]
 [5. 3. 0.]
 [6. 3. 0.]
 [7. 3. 0.]
 [0. 4. 0.]
 [1. 4. 0.]
 [2. 4. 0.]
 [3. 4. 0.]
 [4. 4. 0.]
 [5. 4. 0.]
 [6. 4. 0.]
 [7. 4. 0.]
 [0. 5. 0.]
 [1. 5. 0.]
 [2. 5. 0.]
 [3. 5. 0.]
 [4. 5. 0.]
 [5. 5. 0.]
 [6. 5. 0.]
 [7. 5. 0.]]


In [ ]:

# obyekt kordinatalarini va rasmlarni saqlash uchun bo`sh listlar
objpoints = [] # 3d world kordinatalari sistemasi bo`yicha 3d nuqta. Bizning xolatda z har doim 0 ga teng
imgpoints = [] # 2d image plandagi nuqta

# Rasmlarning listini olib olamiz
images = glob.glob('calibration_wide/GO*.jpg')

# listdagi rasmlardan bir ma bir shaxmat doskani izlaymiz
for idx, fname in enumerate(images):
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # bu operator edge detectorlardan foydalanib shaxmat doskani topadi, va aynan shu edge detectionni yaxshiroq amalga oshirlishligi
    # uchun ham shaxmat doskasi ishlatiladi.
    ret, corners = cv2.findChessboardCorners(gray, (8,6), None)

    # agar topilsa ularni listga qo`shib boradi
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # topilgan shaxmat doskasining burchaklarini chizish
        cv2.drawChessboardCorners(img, (8,6), corners, ret)
        cv2.imshow('img', img)
        cv2.waitKey(500)

cv2.destroyAllWindows()

In [ ]:
import pickle
%matplotlib inline

# Endi esa distorted image ga test qilib ko`ramiz ::
img = cv2.imread('calibration_wide/test_image.jpg')
img_size = (img.shape[1], img.shape[0])

# Camera calibration orqali 
# 0 ret value ;
# 1 camera matrix; 
# 2 distortion matrix; 
# 3 rotation matrix 
# 4 transaction matrix lar olinadi
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size,None,None)


dst = cv2.undistort(img, mtx, dist, None, mtx)
cv2.imwrite('calibration_wide/test_undist.jpg',dst)

# Save the camera calibration result for later use (we won't worry about rvecs / tvecs)
dist_pickle = {}
dist_pickle["mtx"] = mtx
dist_pickle["dist"] = dist
pickle.dump( dist_pickle, open( "calibration_wide/wide_dist_pickle.p", "wb" ) )
#dst = cv2.cvtColor(dst, cv2.COLOR_BGR2RGB)
# Visualize undistortion
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=30)
ax2.imshow(dst)
ax2.set_title('Undistorted Image', fontsize=30)